# Task 1 - Exploration of customer purchasing behavior

In [9]:
# importing libraries
import warnings
warnings.filterwarnings("ignore")
#Data Manipulation and Treatment
import numpy as np
import pandas as pd
from datetime import datetime
#Plotting and Visualizations
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from scipy import stats
import itertools
#dvc
import dvc.api
import mlflow


In [11]:
#utils 
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from plotsClass import Plot
from logger import App_Logger

In [12]:
logger = App_Logger("data_exploration.log").get_app_logger()
plot = Plot()

In [13]:
pd.set_option('max_column', None)
pd.set_option('display.float_format',lambda x:'%5f'%x)

# Data overview and cleaning data

In [14]:
# changing string to date
def str_to_date(date):
      return datetime.strptime(date, '%Y-%m-%d').date()

<h3> Load the datasets <h3>

In [17]:
# reading a dataframe
df_train = pd.read_csv("../data/train.csv",sep=',', parse_dates=['Date'],date_parser=str_to_date, low_memory = False)

In [18]:
# reading the store data
df_store = pd.read_csv("../data/store.csv",sep=',',low_memory = False)

In [19]:
# reading the test data
df_test = pd.read_csv("../data/test.csv",sep=',', parse_dates=['Date'], date_parser=str_to_date,
low_memory = False)

<h3>A quick view at the data on hand<h3>

In [20]:
df_train.head() 

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [21]:
df_train.dtypes,print (f'The Train dataset has {str(df_train.shape[0])} Rows and {str(df_train.shape[1])} Variables')

The Train dataset has 1017209 Rows and 9 Variables


(Store                     int64
 DayOfWeek                 int64
 Date             datetime64[ns]
 Sales                     int64
 Customers                 int64
 Open                      int64
 Promo                     int64
 StateHoliday             object
 SchoolHoliday             int64
 dtype: object,
 None)

<h3>Calculating how many missing fields each variable has<h3>

In [22]:
df_train.count(0)/df_train.shape[0] * 100

Store           100.000000
DayOfWeek       100.000000
Date            100.000000
Sales           100.000000
Customers       100.000000
Open            100.000000
Promo           100.000000
StateHoliday    100.000000
SchoolHoliday   100.000000
dtype: float64

### no missing value found on train data set but a closer look at the Train set is needed

In [30]:
class cleaningTrainData :

    def __init__(self, df):
        
        self.df = df
    # changing string to date
    def drop (self,df):
      df=df.drop(df[(df.Open == 0) & (df.Sales == 0)].index)
      df= df.reset_index(drop=True)
      return df
    def dropone (self,df):
      df=df.drop(df[(df.Open == 1) & (df.Sales == 0)].index)
      df= df.reset_index(drop=True) 
      return df


In [31]:
cleandata = cleaningTrainData(df_train)

### let us see Stores Closed (which means 0 customers and 0 sales) on Certain days:

In [25]:
print (f"-Over those two years, {df_train[(df_train.Open == 0)].count()[0]} is the number of times that different stores closed on given days.")
print ()
print (f"-From those closed events, {df_train[(df_train.Open == 0) & (df_train.SchoolHoliday == 1)&(df_train.StateHoliday == '0') ].count()[0]} times occured because there was a school holiday. ")
print ()
print ("-And {} times it occured because of either a bank holiday or easter or christmas.".format(df_train[(df_train.Open == 0) &
         ((df_train.StateHoliday == 'a') |
          (df_train.StateHoliday == 'b') | 
          (df_train.StateHoliday == 'c'))].count()[0]))
print ()
print ("-But interestingly enough, {} times those shops closed on days for no apparent reason when no holiday was announced. In fact, those closings were done with no pattern whatsoever and in this case from 2013 to 2015 at almost any month and any day.".format(df_train[(df_train.Open == 0) &
         (df_train.StateHoliday == "0")
         &(df_train.SchoolHoliday == 0)].count()[0]))
print ()

-Over those two years, 172817 is the number of times that different stores closed on given days.

-From those closed events, 3067 times occured because there was a school holiday. 

-And 30140 times it occured because of either a bank holiday or easter or christmas.

-But interestingly enough, 139610 times those shops closed on days for no apparent reason when no holiday was announced. In fact, those closings were done with no pattern whatsoever and in this case from 2013 to 2015 at almost any month and any day.



### since we don't want to bias our models to consider those exceptions, the best solution here is to get rid of closed stores and prevent the models to train on them and get false guidance.

### In this case we will analyse only open stores since a close store yield a profit of 0.

In [32]:
df_train=cleandata.drop(df_train)

In [33]:
print ("Our new training set has now {} rows ".format(df_train.shape[0]))

Our new training set has now 844392 rows 


## checking any outliers in the distribution of Sales and Customers in the train set

## 1)sales

In [35]:
df_train.Sales.describe()

count   844392.000000
mean      6955.514291
std       3104.214680
min          0.000000
25%       4859.000000
50%       6369.000000
75%       8360.000000
max      41551.000000
Name: Sales, dtype: float64

df_train.Sales.describe()

### we see here a minimum of 0 which means some stores even opened got 0 sales on some days. since that can't happen we will drop it

In [36]:
df_train=cleandata.dropone(df_train)

### An important metric to always check when looking at a distribution is how the mean compares to the median and how close are they from each other. As we see here a mean of 6955 versus 6369 in median is a very good sign that there are no extravagant values affecting the general distribution of Sales.

## 2) customer

In [37]:
df_train.Customers.describe()    

count   844338.000000
mean       762.777166
std        401.194153
min          8.000000
25%        519.000000
50%        676.000000
75%        893.000000
max       7388.000000
Name: Customers, dtype: float64

## Here there is huge difference between mean and median. This is due to a huge amount of customers in a store,When there is a big promotion going on. 

In [38]:
df_store.head() 

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.000000,9.000000,2008.000000,0,NaN,NaN,NaN
1,2,a,a,570.000000,11.000000,2007.000000,1,13.000000,2010.000000,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.000000,12.000000,2006.000000,1,14.000000,2011.000000,"Jan,Apr,Jul,Oct"
3,4,c,c,620.000000,9.000000,2009.000000,0,NaN,NaN,NaN
4,5,a,a,29910.000000,4.000000,2015.000000,0,NaN,NaN,NaN


In [39]:
df_store.dtypes,print (f'The Store dataset has {str(df_store.shape[0])} Rows and {str(df_store.shape[1])} Variables')

The Store dataset has 1115 Rows and 10 Variables


(Store                          int64
 StoreType                     object
 Assortment                    object
 CompetitionDistance          float64
 CompetitionOpenSinceMonth    float64
 CompetitionOpenSinceYear     float64
 Promo2                         int64
 Promo2SinceWeek              float64
 Promo2SinceYear              float64
 PromoInterval                 object
 dtype: object,
 None)